In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
# Set end_period as a date object.
end_period = pd.to_datetime("2025-11-27").date()

In [3]:
# # Path to the .xls file
# file_path = r"C:\Users\leere\OneDrive\Desktop\RAW DATA\ml_goals.xls"
#
# # Load the Excel file into a DataFrame
# df = pd.read_excel(file_path)
#
# # Display the first few rows of the DataFrame
# df.head()


In [4]:
import pandas as pd
from pathlib import Path
import importlib.util


def load_excel_auto_convert(file_path: str, *, convert_to_xlsx: bool = True) -> pd.DataFrame:
    """
    Load an Excel file robustly.

    Behaviour:
      - .xlsx / .xlsm → normal pd.read_excel
      - .xls →
          * try engine="calamine" first (python-calamine)
          * if that fails, try engine="xlrd"
          * optionally save a .xlsx copy IF openpyxl is installed
      - anything else → try pd.read_csv
    """
    path = Path(file_path)

    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")

    suffix = path.suffix.lower()

    # Modern Excel formats
    if suffix in {".xlsx", ".xlsm"}:
        return pd.read_excel(path)

    # Legacy .xls handling
    if suffix == ".xls":
        df = None
        last_error = None

        # 1) Prefer calamine
        for engine in ("calamine", "xlrd"):
            try:
                df = pd.read_excel(path, engine=engine)
                print(f"Loaded {path.name} using engine='{engine}'.")
                break
            except Exception as e:
                last_error = e
                print(f"Engine '{engine}' failed: {e}")

        if df is None:
            raise RuntimeError(
                f"Failed to read legacy .xls file '{path}'. "
                f"Last error: {last_error}\n"
                f"Install python-calamine (and optionally xlrd), "
                f"or open the file in Excel and re-save as .xlsx."
            ) from last_error

        # 2) Optionally: convert to .xlsx IF openpyxl is available
        if convert_to_xlsx:
            has_openpyxl = importlib.util.find_spec("openpyxl") is not None
            if not has_openpyxl:
                print(
                    "openpyxl is not installed – skipping .xlsx export. "
                    "Data is loaded into a DataFrame and ready to use."
                )
            else:
                try:
                    xlsx_path = path.with_suffix(".xlsx")
                    df.to_excel(xlsx_path, index=False)
                    print(f"Converted {path.name} → {xlsx_path.name}")
                except Exception as e:
                    # Don't crash just because the export failed
                    print(f"Could not write .xlsx copy: {e}")

        return df

    # Fallback: non-Excel → assume CSV
    print(f"{path.name} is not an Excel file, trying CSV reader.")
    return pd.read_csv(path)

file_path = r"C:\Users\leere\OneDrive\Desktop\RAW DATA\ml_goals.xls"
df = load_excel_auto_convert(file_path)

df.head()


Loaded ml_goals.xls using engine='calamine'.
openpyxl is not installed – skipping .xlsx export. Data is loaded into a DataFrame and ready to use.


,country,sezonul,datameci,orameci,etapa,txtechipa1,txtechipa2,scor1,scor2,scorp1,...,yellowa2,ballph,ballph1,ballph2,ballpa,ballpa1,ballpa2,stare,codechipa1,codechipa2
0,Mex1,25,2024-07-05,2345,1,Puebla,Santos Laguna,1,0,0,...,2,40,39,41,60,61,59,J,41017.0,41008.0
1,Mex1,25,2024-07-06,200,1,Queretaro,Tijuana de Caliente,1,2,0,...,0,37,33,41,63,67,59,J,41011.0,41006.0
2,Mex1,25,2024-07-06,410,1,Juarez,Atlas,2,2,1,...,0,55,66,44,45,34,56,J,41020.0,41002.0
3,Mex1,25,2024-07-07,0,1,San Luis,Club America,2,1,1,...,1,37,41,33,63,59,67,J,41019.0,41012.0
4,Mex1,25,2024-07-07,200,1,G. Chivas,Toluca,0,0,0,...,2,52,46,58,48,54,42,J,41013.0,41007.0


In [5]:
column_dict = {
    "country": "country",
    "league": "league",
    "sezonul": "season",
    "datameci": "date",
    "orameci": "ko_time",
    "etapa": "round",
    "txtechipa1": "home_team",
    "txtechipa2": "away_team",
    "scor1": "home_goals_ft",
    "scor2": "away_goals_ft",
    "scorp1": "home_goals_ht",
    "scorp2": "away_goals_ht",
    "place1": "home_team_place_total",
    "place1a": "home_team_place_home",
    "place2": "away_team_place_total",
    "place2d": "away_team_place_away",
    "cotaa": "home_odds",
    "cotae": "draw_odds",
    "cotad": "away_odds",
    # "cotao0": "",
    # "cotao1": "",
    "cotao": "over_25_odds",
    # "cotao3": "",
    # "cotao4": "",
    # "cotau0": "",
    # "cotau1": "",
    "cotau": "under_25_odds",
    # "cotau3": "",
    # "cotau4": "",
    # "gg": "",
    # "ng": "",
    "elohomeo": "elo_home",
    "eloawayo": "elo_away",
    "formah": "form_home",
    "formaa": "form_away",
    "suth": "shots_home",
    "suth1": "shots_home_1h",
    "suth2": "shots_home_2h",
    "suta": "shots_away",
    "suta1": "shots_away_1h",
    "suta2": "shots_away_2h",
    "sutht": "shots_on_target_home",
    "sutht1": "shots_on_target_home_1h",
    "sutht2": "shots_on_target_home_2h",
    "sutat": "shots_on_target_away",
    "sutat1": "shots_on_target_away_1h",
    "sutat2": "shots_on_target_away_2h",
    "corh": "corners_home",
    "corh1": "corners_home_1h",
    "corh2": "corners_home_2h",
    "cora": "corners_away",
    "cora1": "corners_away_1h",
    "cora2": "corners_away_2h",
    "foulsh": "fouls_home",
    "foulsh1": "fouls_home_1h",
    "foulsh2": "fouls_home_2h",
    "foulsa": "fouls_away",
    "foulsa1": "fouls_away_1h",
    "foulsa2": "fouls_away_2h",
    "yellowh": "yellow_cards_home",
    "yellowh1": "yellow_cards_home_1h",
    "yellowh2": "yellow_cards_home_2h",
    "yellowa": "yellow_cards_away",
    "yellowa1": "yellow_cards_away_1h",
    "yellowa2": "yellow_cards_away_2h",
    "ballph": "possession_home",
    "ballph1": "possession_home_1h",
    "ballph2": "possession_home_2h",
    "ballpa": "possession_away",
    "ballpa1": "possession_away_1h",
    "ballpa2": "possession_away_2h",
    "gsh": "goals_scored_total_home",
    "gch": "goals_conceded_total_home",
    "gsa": "goals_scored_total_away",
    "gca": "goals_conceded_total_away",
    # "stare": "",
    # "codechipa1": "",
    # "codechipa2": ""
}

df = df.rename(columns=column_dict).filter(items=column_dict.values())
data = df.copy()
data

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,yellow_cards_home_2h,yellow_cards_away,yellow_cards_away_1h,yellow_cards_away_2h,possession_home,possession_home_1h,possession_home_2h,possession_away,possession_away_1h,possession_away_2h
0,Mex1,25,2024-07-05,2345,1,Puebla,Santos Laguna,1,0,0,...,1,3,1,2,40,39,41,60,61,59
1,Mex1,25,2024-07-06,200,1,Queretaro,Tijuana de Caliente,1,2,0,...,2,0,0,0,37,33,41,63,67,59
2,Mex1,25,2024-07-06,410,1,Juarez,Atlas,2,2,1,...,1,2,2,0,55,66,44,45,34,56
3,Mex1,25,2024-07-07,0,1,San Luis,Club America,2,1,1,...,3,2,1,1,37,41,33,63,59,67
4,Mex1,25,2024-07-07,200,1,G. Chivas,Toluca,0,0,0,...,0,2,0,2,52,46,58,48,54,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25099,Spa2,26,2026-05-31,1700,42,Granada,Gijon,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25100,Spa2,26,2026-05-31,1700,42,Leganes,Mirandes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25101,Spa2,26,2026-05-31,1700,42,Santander,Cadiz,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25102,Spa2,26,2026-05-31,1700,42,R. Sociedad B,Leonesa,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Convert 'date' column to datetime object
data['date'] = pd.to_datetime(data['date'], format="%d/%m/%Y", errors='coerce')

# Order by date
data = data.sort_values(by='date')

# Filter out future dates (ensure data does not go beyond today)
today = datetime.today().date()
data = data[data['date'].dt.date <= end_period]

# Create a mask for matches that have been played (i.e. date is less than today)
played_mask = data['date'].dt.date < today

# Calculate home points for played matches only.
data.loc[played_mask, 'points_home'] = np.where(
    data.loc[played_mask, 'home_goals_ft'] > data.loc[played_mask, 'away_goals_ft'], 3,
    np.where(data.loc[played_mask, 'home_goals_ft'] == data.loc[played_mask, 'away_goals_ft'], 1, 0)
)

# Calculate away points for played matches only.
data.loc[played_mask, 'points_away'] = np.where(
    data.loc[played_mask, 'away_goals_ft'] > data.loc[played_mask, 'home_goals_ft'], 3,
    np.where(data.loc[played_mask, 'away_goals_ft'] == data.loc[played_mask, 'home_goals_ft'], 1, 0)
)

In [7]:
data

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,yellow_cards_away_1h,yellow_cards_away_2h,possession_home,possession_home_1h,possession_home_2h,possession_away,possession_away_1h,possession_away_2h,points_home,points_away
0,Mex1,25,2024-07-05,2345,1,Puebla,Santos Laguna,1,0,0,...,1,2,40,39,41,60,61,59,3.0,0.0
1,Mex1,25,2024-07-06,200,1,Queretaro,Tijuana de Caliente,1,2,0,...,0,0,37,33,41,63,67,59,0.0,3.0
2,Mex1,25,2024-07-06,410,1,Juarez,Atlas,2,2,1,...,2,0,55,66,44,45,34,56,1.0,1.0
3,Mex1,25,2024-07-07,0,1,San Luis,Club America,2,1,1,...,1,1,37,41,33,63,59,67,3.0,0.0
4,Mex1,25,2024-07-07,200,1,G. Chivas,Toluca,0,0,0,...,0,2,52,46,58,48,54,42,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18792,Eng2,26,2025-11-26,1945,17,Blackburn,QPR,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
18790,Eng2,26,2025-11-26,1945,17,Millwall,Sheffield Wed,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
18794,Eng2,26,2025-11-26,1945,17,Wrexham,Bristol City,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
18800,Eng4,26,2025-11-27,2000,18,Grimsby,Tranmere,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [8]:
# # # Assign points based on match results
# # data["points_home"] = data.apply(lambda row: 3 if row["home_goals_ft"] > row["away_goals_ft"]
# # else (1 if row["home_goals_ft"] == row["away_goals_ft"] else 0), axis=1)
# #
# # data["points_away"] = data.apply(lambda row: 3 if row["away_goals_ft"] > row["home_goals_ft"]
# # else (1 if row["away_goals_ft"] == row["home_goals_ft"] else 0), axis=1)
#
# # =============================================================================
# # 1. Data Preparation: Build Team-Level Data (Home & Away)
# # =============================================================================
# # Prepare home records.
# home_df = data[['country', 'season', 'date', 'home_team', 'away_team',
#                 'home_goals_ft', 'away_goals_ft', 'home_goals_ht', 'away_goals_ht',
#                 'shots_home', 'shots_home_1h', 'shots_home_2h',
#                 'shots_on_target_home', 'shots_on_target_home_1h', 'shots_on_target_home_2h',
#                 'corners_home', 'corners_home_1h', 'corners_home_2h']].copy()
# home_df.rename(columns={
#     'home_team': 'Team',
#     'away_team': 'Opponent',
#     'home_goals_ft': 'GoalsScored',
#     'away_goals_ft': 'GoalsConceded',
#     'home_goals_ht': 'FirstHalfGoalsScored',
#     'away_goals_ht': 'FirstHalfGoalsConceded',
#     'shots_home': 'Shots',
#     'shots_home_1h': 'Shots_1h',
#     'shots_home_2h': 'Shots_2h',
#     'shots_on_target_home': 'ShotsOnTarget',
#     'shots_on_target_home_1h': 'ShotsOnTarget_1h',
#     'shots_on_target_home_2h': 'ShotsOnTarget_2h',
#     'corners_home': 'Corners',
#     'corners_home_1h': 'Corners_1h',
#     'corners_home_2h': 'Corners_2h'
# }, inplace=True)
# home_df['is_home'] = 1
#
# # Prepare away records.
# away_df = data[['country', 'season', 'date', 'away_team', 'home_team',
#                 'away_goals_ft', 'home_goals_ft', 'away_goals_ht', 'home_goals_ht',
#                 'shots_away', 'shots_away_1h', 'shots_away_2h',
#                 'shots_on_target_away', 'shots_on_target_away_1h', 'shots_on_target_away_2h',
#                 'corners_away', 'corners_away_1h', 'corners_away_2h']].copy()
# away_df.rename(columns={
#     'away_team': 'Team',
#     'home_team': 'Opponent',
#     'away_goals_ft': 'GoalsScored',
#     'home_goals_ft': 'GoalsConceded',
#     'away_goals_ht': 'FirstHalfGoalsScored',
#     'home_goals_ht': 'FirstHalfGoalsConceded',
#     'shots_away': 'Shots',
#     'shots_away_1h': 'Shots_1h',
#     'shots_away_2h': 'Shots_2h',
#     'shots_on_target_away': 'ShotsOnTarget',
#     'shots_on_target_away_1h': 'ShotsOnTarget_1h',
#     'shots_on_target_away_2h': 'ShotsOnTarget_2h',
#     'corners_away': 'Corners',
#     'corners_away_1h': 'Corners_1h',
#     'corners_away_2h': 'Corners_2h'
# }, inplace=True)
# away_df['is_home'] = 0
#
# # Combine both.
# team_df = pd.concat([home_df, away_df], ignore_index=True)
# team_df.sort_values(by=['country', 'season', 'Team', 'date'], inplace=True)
#
# # Define rolling window sizes.
# window_long = 5   # for long-term trends
# window_short = 3  # for short-term momentum
#
# # =============================================================================
# # 2. Rolling Feature Computation Functions
# # =============================================================================
# def compute_slope(x):
#     """Compute slope using simple linear regression."""
#     if len(x) < 2:
#         return np.nan
#     xs = np.arange(len(x))
#     return np.polyfit(xs, x, 1)[0]
#
# def compute_rolling_features_metric(df_sub, full_col, first_half_col, prefix):
#     """
#     Compute rolling features for a given metric.
#     Returns a DataFrame of new columns.
#     """
#     new_cols = {}
#     # Full-match features.
#     new_cols[f'{prefix}_Rolling_{full_col}_Mean'] = df_sub[full_col].rolling(window=window_long, min_periods=1).mean().shift(1)
#     new_cols[f'{prefix}_Rolling_{full_col}_Std']  = df_sub[full_col].rolling(window=window_long, min_periods=1).std().shift(1)
#     new_cols[f'{prefix}_Rolling_{full_col}_Mean_Short'] = df_sub[full_col].rolling(window=window_short, min_periods=1).mean().shift(1)
#     new_cols[f'{prefix}_Momentum_{full_col}'] = new_cols[f'{prefix}_Rolling_{full_col}_Mean_Short'] - new_cols[f'{prefix}_Rolling_{full_col}_Mean']
#     new_cols[f'{prefix}_Trend_Slope_{full_col}'] = df_sub[full_col].rolling(window=window_long, min_periods=2).apply(compute_slope, raw=True).shift(1)
#     # First-half features.
#     new_cols[f'{prefix}_Rolling_{first_half_col}_Mean'] = df_sub[first_half_col].rolling(window=window_long, min_periods=1).mean().shift(1)
#     new_cols[f'{prefix}_Rolling_{first_half_col}_Std']  = df_sub[first_half_col].rolling(window=window_long, min_periods=1).std().shift(1)
#     new_cols[f'{prefix}_Rolling_{first_half_col}_Mean_Short'] = df_sub[first_half_col].rolling(window=window_short, min_periods=1).mean().shift(1)
#     new_cols[f'{prefix}_Momentum_{first_half_col}'] = new_cols[f'{prefix}_Rolling_{first_half_col}_Mean_Short'] - new_cols[f'{prefix}_Rolling_{first_half_col}_Mean']
#     new_cols[f'{prefix}_Trend_Slope_{first_half_col}'] = df_sub[first_half_col].rolling(window=window_long, min_periods=2).apply(compute_slope, raw=True).shift(1)
#     return pd.DataFrame(new_cols, index=df_sub.index)
#
# def add_rolling_features_split(group):
#     """Compute overall, home-, and away-specific rolling features plus outcome percentages."""
#     group = group.sort_values(by='date').reset_index(drop=True)
#
#     # Overall features.
#     overall_features = pd.concat([
#         compute_rolling_features_metric(group, 'GoalsScored', 'FirstHalfGoalsScored', 'Overall'),
#         compute_rolling_features_metric(group, 'Shots', 'Shots_1h', 'Overall'),
#         compute_rolling_features_metric(group, 'Corners', 'Corners_1h', 'Overall'),
#         compute_rolling_features_metric(group, 'ShotsOnTarget', 'ShotsOnTarget_1h', 'Overall')
#     ], axis=1)
#     group = pd.concat([group, overall_features], axis=1)
#
#     home_mask = group['is_home'] == 1
#     away_mask = group['is_home'] == 0
#
#     # Home-specific.
#     if home_mask.any():
#         home_feats = pd.concat([
#             compute_rolling_features_metric(group.loc[home_mask], 'GoalsScored', 'FirstHalfGoalsScored', 'Home'),
#             compute_rolling_features_metric(group.loc[home_mask], 'Shots', 'Shots_1h', 'Home'),
#             compute_rolling_features_metric(group.loc[home_mask], 'Corners', 'Corners_1h', 'Home'),
#             compute_rolling_features_metric(group.loc[home_mask], 'ShotsOnTarget', 'ShotsOnTarget_1h', 'Home')
#         ], axis=1)
#         group.loc[home_mask, home_feats.columns] = home_feats
#
#     # Away-specific.
#     if away_mask.any():
#         away_feats = pd.concat([
#             compute_rolling_features_metric(group.loc[away_mask], 'GoalsScored', 'FirstHalfGoalsScored', 'Away'),
#             compute_rolling_features_metric(group.loc[away_mask], 'Shots', 'Shots_1h', 'Away'),
#             compute_rolling_features_metric(group.loc[away_mask], 'Corners', 'Corners_1h', 'Away'),
#             compute_rolling_features_metric(group.loc[away_mask], 'ShotsOnTarget', 'ShotsOnTarget_1h', 'Away')
#         ], axis=1)
#         group.loc[away_mask, away_feats.columns] = away_feats
#
#     # Additional outcome percentages for goals.
#     thresh_dict = {}
#     for thresh in [1.5, 2.5, 3.5]:
#         thresh_dict[f'Overall_Percent_Over_{thresh}'] = group['GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
#         thresh_dict[f'Overall_Rolling5_Percent_Over_{thresh}'] = group['GoalsScored'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
#         if home_mask.any():
#             thresh_dict[f'Home_Percent_Over_{thresh}'] = group.loc[home_mask, 'GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
#             thresh_dict[f'Home_Rolling5_Percent_Over_{thresh}'] = group.loc[home_mask, 'GoalsScored'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
#         if away_mask.any():
#             thresh_dict[f'Away_Percent_Over_{thresh}'] = group.loc[away_mask, 'GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
#             thresh_dict[f'Away_Rolling5_Percent_Over_{thresh}'] = group.loc[away_mask, 'GoalsScored'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
#     group = pd.concat([group, pd.DataFrame(thresh_dict, index=group.index)], axis=1)
#
#     # Outcome percentages for goals.
#     outcome_dict = {}
#     for thresh in [0.5, 1.5, 2.5, 3.5]:
#         outcome_dict[f'TeamPct_Over_{thresh}'] = group['GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
#         if home_mask.any():
#             outcome_dict[f'Home_TeamPct_Over_{thresh}'] = group.loc[home_mask, 'GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
#         if away_mask.any():
#             outcome_dict[f'Away_TeamPct_Over_{thresh}'] = group.loc[away_mask, 'GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
#     group = pd.concat([group, pd.DataFrame(outcome_dict, index=group.index)], axis=1)
#
#     # Outcome percentages for corners.
#     corners_thresh = [3.5, 4.5, 5.5, 6.5]
#     corners_dict = {}
#     for thresh in corners_thresh:
#         corners_dict[f'CornersPct_Over_{thresh}'] = group['Corners'].gt(thresh).shift(1).expanding(min_periods=1).mean()
#         corners_dict[f'CornersRolling5Pct_Over_{thresh}'] = group['Corners'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
#         if home_mask.any():
#             corners_dict[f'Home_CornersPct_Over_{thresh}'] = group.loc[home_mask, 'Corners'].gt(thresh).shift(1).expanding(min_periods=1).mean()
#             corners_dict[f'Home_CornersRolling5Pct_Over_{thresh}'] = group.loc[home_mask, 'Corners'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
#         if away_mask.any():
#             corners_dict[f'Away_CornersPct_Over_{thresh}'] = group.loc[away_mask, 'Corners'].gt(thresh).shift(1).expanding(min_periods=1).mean()
#             corners_dict[f'Away_CornersRolling5Pct_Over_{thresh}'] = group.loc[away_mask, 'Corners'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
#     group = pd.concat([group, pd.DataFrame(corners_dict, index=group.index)], axis=1)
#
#     return group
#
# # Apply group-wise computations.
# team_df = team_df.groupby(['country', 'season', 'Team'], group_keys=False).apply(add_rolling_features_split).reset_index(drop=True)
# team_df = team_df.copy()  # ensure defragmentation
#
# # =============================================================================
# # 3. Compute Team-Level Corners Outcome Features (from Match Data)
# # =============================================================================
# # Build a match-level DataFrame for corners outcomes.
# match_df = data.copy()
# match_df['Total_Corners'] = match_df['corners_home'] + match_df['corners_away']
# match_df.sort_values(by=['country', 'season', 'date'], inplace=True)
#
# # Create a team perspective by combining home and away records.
# home_matches = match_df[['country', 'season', 'date', 'home_team', 'Total_Corners']].copy()
# home_matches.rename(columns={'home_team': 'Team'}, inplace=True)
# away_matches = match_df[['country', 'season', 'date', 'away_team', 'Total_Corners']].copy()
# away_matches.rename(columns={'away_team': 'Team'}, inplace=True)
# team_corners_matches = pd.concat([home_matches, away_matches], ignore_index=True)
# team_corners_matches.sort_values(by=['country', 'season', 'Team', 'date'], inplace=True)
#
# # For thresholds 9.5, 10.5, and 11.5, compute season-level and rolling percentages.
# for thr in [9.5, 10.5, 11.5]:
#     indicator = f'Over_{thr}'
#     team_corners_matches[indicator] = (team_corners_matches['Total_Corners'] > thr).astype(int)
#     team_corners_matches[f'SeasonPct_{indicator}'] = team_corners_matches.groupby(
#         ['country', 'season', 'Team']
#     )[indicator].transform(lambda x: x.shift(1).expanding(min_periods=1).mean())
#     team_corners_matches[f'Rolling5Pct_{indicator}'] = team_corners_matches.groupby(
#         ['country', 'season', 'Team']
#     )[indicator].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())
#
# # Select only the keys and outcome columns for merging.
# cols_to_merge = ['country', 'season', 'date', 'Team',
#                  'SeasonPct_Over_9.5', 'Rolling5Pct_Over_9.5',
#                  'SeasonPct_Over_10.5', 'Rolling5Pct_Over_10.5',
#                  'SeasonPct_Over_11.5', 'Rolling5Pct_Over_11.5']
#
# # Merge the corners outcome features into team_df.
# team_df = team_df.merge(team_corners_matches[cols_to_merge],
#                         on=['country', 'season', 'date', 'Team'],
#                         how='left')
#
# # =============================================================================
# # 4. Process Home and Away Features for Match-Level Merging
# # =============================================================================
# # -- Home-Team Process --
# home_subset = team_df[team_df['is_home'] == 1].copy()
# home_subset.drop(columns=['Opponent'], inplace=True)
# home_subset.rename(columns={'Team': 'home_team'}, inplace=True)
# home_key = ['country', 'season', 'date', 'home_team', 'is_home']
# # Include features starting with Overall_, Home_, SeasonPct_Over_, or Rolling5Pct_Over_
# home_feats = [col for col in home_subset.columns if col not in home_key and
#               (col.startswith("Overall_") or col.startswith("Home_") or
#                col.startswith("SeasonPct_Over_") or col.startswith("Rolling5Pct_Over_"))]
# home_features = home_subset[home_key + home_feats].copy()
# def clean_home_name(col):
#     return "home_" + (col[len("Home_"):] if col.startswith("Home_") else col)
# home_features.rename(columns={col: clean_home_name(col) for col in home_feats}, inplace=True)
#
# # -- Away-Team Process --
# away_subset = team_df[team_df['is_home'] == 0].copy()
# away_subset.drop(columns=['Opponent'], inplace=True)
# away_subset.rename(columns={'Team': 'away_team'}, inplace=True)
# away_key = ['country', 'season', 'date', 'away_team', 'is_home']
# away_feats = [col for col in away_subset.columns if col not in away_key and
#               (col.startswith("Overall_") or col.startswith("Away_") or
#                col.startswith("SeasonPct_Over_") or col.startswith("Rolling5Pct_Over_"))]
# away_features = away_subset[away_key + away_feats].copy()
# def clean_away_name(col):
#     return "away_" + (col[len("Away_"):] if col.startswith("Away_") else col)
# away_features.rename(columns={col: clean_away_name(col) for col in away_feats}, inplace=True)
#
# # =============================================================================
# # 5. Merge Home and Away Features into the Match-Level DataFrame
# # =============================================================================
# # Start with the original match data.
# match_merge_df = data.copy()
# # Merge home features.
# match_merge_df = match_merge_df.merge(home_features, on=['country', 'season', 'date', 'home_team'], how='left')
# # Merge away features.
# match_merge_df = match_merge_df.merge(away_features, on=['country', 'season', 'date', 'away_team'], how='left')
#
# # (Optional) Display a sample.
# #print(match_merge_df.head())


In [9]:
# =============================================================================
# 1. Data Preparation: Build Team-Level Data (Home & Away)
# =============================================================================
# Prepare home records.
home_df = data[['country', 'season', 'date', 'home_team', 'away_team',
                'home_goals_ft', 'away_goals_ft', 'home_goals_ht', 'away_goals_ht',
                'shots_home', 'shots_home_1h', 'shots_home_2h',
                'shots_on_target_home', 'shots_on_target_home_1h', 'shots_on_target_home_2h',
                'corners_home', 'corners_home_1h', 'corners_home_2h']].copy()
home_df.rename(columns={
    'home_team': 'Team',
    'away_team': 'Opponent',
    'home_goals_ft': 'GoalsScored',
    'away_goals_ft': 'GoalsConceded',
    'home_goals_ht': 'FirstHalfGoalsScored',
    'away_goals_ht': 'FirstHalfGoalsConceded',
    'shots_home': 'Shots',
    'shots_home_1h': 'Shots_1h',
    'shots_home_2h': 'Shots_2h',
    'shots_on_target_home': 'ShotsOnTarget',
    'shots_on_target_home_1h': 'ShotsOnTarget_1h',
    'shots_on_target_home_2h': 'ShotsOnTarget_2h',
    'corners_home': 'Corners',
    'corners_home_1h': 'Corners_1h',
    'corners_home_2h': 'Corners_2h'
}, inplace=True)
home_df['is_home'] = 1

# Prepare away records.
away_df = data[['country', 'season', 'date', 'away_team', 'home_team',
                'away_goals_ft', 'home_goals_ft', 'away_goals_ht', 'home_goals_ht',
                'shots_away', 'shots_away_1h', 'shots_away_2h',
                'shots_on_target_away', 'shots_on_target_away_1h', 'shots_on_target_away_2h',
                'corners_away', 'corners_away_1h', 'corners_away_2h']].copy()
away_df.rename(columns={
    'away_team': 'Team',
    'home_team': 'Opponent',
    'away_goals_ft': 'GoalsScored',
    'home_goals_ft': 'GoalsConceded',
    'away_goals_ht': 'FirstHalfGoalsScored',
    'home_goals_ht': 'FirstHalfGoalsConceded',
    'shots_away': 'Shots',
    'shots_away_1h': 'Shots_1h',
    'shots_away_2h': 'Shots_2h',
    'shots_on_target_away': 'ShotsOnTarget',
    'shots_on_target_away_1h': 'ShotsOnTarget_1h',
    'shots_on_target_away_2h': 'ShotsOnTarget_2h',
    'corners_away': 'Corners',
    'corners_away_1h': 'Corners_1h',
    'corners_away_2h': 'Corners_2h'
}, inplace=True)
away_df['is_home'] = 0

# Combine both.
team_df = pd.concat([home_df, away_df], ignore_index=True)
team_df.sort_values(by=['country', 'season', 'Team', 'date'], inplace=True)

# Define rolling window sizes.
window_long = 5   # for long-term trends
window_short = 3  # for short-term momentum

# =============================================================================
# 2. Rolling Feature Computation Functions
# =============================================================================
def compute_slope(x):
    """Compute slope using simple linear regression."""
    if len(x) < 2:
        return np.nan
    xs = np.arange(len(x))
    return np.polyfit(xs, x, 1)[0]

def compute_rolling_features_metric(df_sub, full_col, first_half_col, prefix):
    """
    Compute rolling features for a given metric.
    Returns a DataFrame of new columns.
    """
    new_cols = {}
    # Full-match features.
    new_cols[f'{prefix}_Rolling_{full_col}_Mean'] = df_sub[full_col].rolling(window=window_long, min_periods=1).mean().shift(1)
    new_cols[f'{prefix}_Rolling_{full_col}_Std']  = df_sub[full_col].rolling(window=window_long, min_periods=1).std().shift(1)
    new_cols[f'{prefix}_Rolling_{full_col}_Mean_Short'] = df_sub[full_col].rolling(window=window_short, min_periods=1).mean().shift(1)
    new_cols[f'{prefix}_Momentum_{full_col}'] = new_cols[f'{prefix}_Rolling_{full_col}_Mean_Short'] - new_cols[f'{prefix}_Rolling_{full_col}_Mean']
    new_cols[f'{prefix}_Trend_Slope_{full_col}'] = df_sub[full_col].rolling(window=window_long, min_periods=2).apply(compute_slope, raw=True).shift(1)
    # First-half features.
    new_cols[f'{prefix}_Rolling_{first_half_col}_Mean'] = df_sub[first_half_col].rolling(window=window_long, min_periods=1).mean().shift(1)
    new_cols[f'{prefix}_Rolling_{first_half_col}_Std']  = df_sub[first_half_col].rolling(window=window_long, min_periods=1).std().shift(1)
    new_cols[f'{prefix}_Rolling_{first_half_col}_Mean_Short'] = df_sub[first_half_col].rolling(window=window_short, min_periods=1).mean().shift(1)
    new_cols[f'{prefix}_Momentum_{first_half_col}'] = new_cols[f'{prefix}_Rolling_{first_half_col}_Mean_Short'] - new_cols[f'{prefix}_Rolling_{first_half_col}_Mean']
    new_cols[f'{prefix}_Trend_Slope_{first_half_col}'] = df_sub[first_half_col].rolling(window=window_long, min_periods=2).apply(compute_slope, raw=True).shift(1)
    return pd.DataFrame(new_cols, index=df_sub.index)

def add_rolling_features_split(group):
    """Compute overall, home-, and away-specific rolling features plus outcome percentages."""
    group = group.sort_values(by='date').reset_index(drop=True)

    # Overall features.
    overall_features = pd.concat([
        compute_rolling_features_metric(group, 'GoalsScored', 'FirstHalfGoalsScored', 'Overall'),
        compute_rolling_features_metric(group, 'Shots', 'Shots_1h', 'Overall'),
        compute_rolling_features_metric(group, 'Corners', 'Corners_1h', 'Overall'),
        compute_rolling_features_metric(group, 'ShotsOnTarget', 'ShotsOnTarget_1h', 'Overall')
    ], axis=1)
    group = pd.concat([group, overall_features], axis=1)

    home_mask = group['is_home'] == 1
    away_mask = group['is_home'] == 0

    # Home-specific.
    if home_mask.any():
        home_feats = pd.concat([
            compute_rolling_features_metric(group.loc[home_mask], 'GoalsScored', 'FirstHalfGoalsScored', 'Home'),
            compute_rolling_features_metric(group.loc[home_mask], 'Shots', 'Shots_1h', 'Home'),
            compute_rolling_features_metric(group.loc[home_mask], 'Corners', 'Corners_1h', 'Home'),
            compute_rolling_features_metric(group.loc[home_mask], 'ShotsOnTarget', 'ShotsOnTarget_1h', 'Home')
        ], axis=1)
        group.loc[home_mask, home_feats.columns] = home_feats

    # Away-specific.
    if away_mask.any():
        away_feats = pd.concat([
            compute_rolling_features_metric(group.loc[away_mask], 'GoalsScored', 'FirstHalfGoalsScored', 'Away'),
            compute_rolling_features_metric(group.loc[away_mask], 'Shots', 'Shots_1h', 'Away'),
            compute_rolling_features_metric(group.loc[away_mask], 'Corners', 'Corners_1h', 'Away'),
            compute_rolling_features_metric(group.loc[away_mask], 'ShotsOnTarget', 'ShotsOnTarget_1h', 'Away')
        ], axis=1)
        group.loc[away_mask, away_feats.columns] = away_feats

    # Additional outcome percentages for goals.
    thresh_dict = {}
    for thresh in [1.5, 2.5, 3.5]:
        thresh_dict[f'Overall_Percent_Over_{thresh}'] = group['GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
        thresh_dict[f'Overall_Rolling5_Percent_Over_{thresh}'] = group['GoalsScored'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
        if home_mask.any():
            thresh_dict[f'Home_Percent_Over_{thresh}'] = group.loc[home_mask, 'GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
            thresh_dict[f'Home_Rolling5_Percent_Over_{thresh}'] = group.loc[home_mask, 'GoalsScored'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
        if away_mask.any():
            thresh_dict[f'Away_Percent_Over_{thresh}'] = group.loc[away_mask, 'GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
            thresh_dict[f'Away_Rolling5_Percent_Over_{thresh}'] = group.loc[away_mask, 'GoalsScored'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
    group = pd.concat([group, pd.DataFrame(thresh_dict, index=group.index)], axis=1)

    # Outcome percentages for goals.
    outcome_dict = {}
    for thresh in [0.5, 1.5, 2.5, 3.5]:
        outcome_dict[f'TeamPct_Over_{thresh}'] = group['GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
        if home_mask.any():
            outcome_dict[f'Home_TeamPct_Over_{thresh}'] = group.loc[home_mask, 'GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
        if away_mask.any():
            outcome_dict[f'Away_TeamPct_Over_{thresh}'] = group.loc[away_mask, 'GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
    group = pd.concat([group, pd.DataFrame(outcome_dict, index=group.index)], axis=1)

    # Outcome percentages for corners.
    corners_thresh = [3.5, 4.5, 5.5, 6.5]
    corners_dict = {}
    for thresh in corners_thresh:
        corners_dict[f'CornersPct_Over_{thresh}'] = group['Corners'].gt(thresh).shift(1).expanding(min_periods=1).mean()
        corners_dict[f'CornersRolling5Pct_Over_{thresh}'] = group['Corners'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
        if home_mask.any():
            corners_dict[f'Home_CornersPct_Over_{thresh}'] = group.loc[home_mask, 'Corners'].gt(thresh).shift(1).expanding(min_periods=1).mean()
            corners_dict[f'Home_CornersRolling5Pct_Over_{thresh}'] = group.loc[home_mask, 'Corners'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
        if away_mask.any():
            corners_dict[f'Away_CornersPct_Over_{thresh}'] = group.loc[away_mask, 'Corners'].gt(thresh).shift(1).expanding(min_periods=1).mean()
            corners_dict[f'Away_CornersRolling5Pct_Over_{thresh}'] = group.loc[away_mask, 'Corners'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
    group = pd.concat([group, pd.DataFrame(corners_dict, index=group.index)], axis=1)

    return group

# Apply group-wise computations.
team_df = team_df.groupby(['country', 'season', 'Team'], group_keys=False).apply(add_rolling_features_split).reset_index(drop=True)
team_df = team_df.copy()  # ensure defragmentation

# =============================================================================
# 3. Compute Team-Level Corners Outcome Features (from Match Data)
# =============================================================================
# Build a match-level DataFrame for corners outcomes.
match_df = data.copy()
match_df['Total_Corners'] = match_df['corners_home'] + match_df['corners_away']
match_df.sort_values(by=['country', 'season', 'date'], inplace=True)

# Create a team perspective by combining home and away records.
home_matches = match_df[['country', 'season', 'date', 'home_team', 'Total_Corners']].copy()
home_matches.rename(columns={'home_team': 'Team'}, inplace=True)
away_matches = match_df[['country', 'season', 'date', 'away_team', 'Total_Corners']].copy()
away_matches.rename(columns={'away_team': 'Team'}, inplace=True)
team_corners_matches = pd.concat([home_matches, away_matches], ignore_index=True)
team_corners_matches.sort_values(by=['country', 'season', 'Team', 'date'], inplace=True)

# For thresholds 9.5, 10.5, and 11.5, compute season-level and rolling percentages.
for thr in [9.5, 10.5, 11.5]:
    indicator = f'Over_{thr}'
    team_corners_matches[indicator] = (team_corners_matches['Total_Corners'] > thr).astype(int)
    team_corners_matches[f'SeasonPct_{indicator}'] = team_corners_matches.groupby(
        ['country', 'season', 'Team']
    )[indicator].transform(lambda x: x.shift(1).expanding(min_periods=1).mean())
    team_corners_matches[f'Rolling5Pct_{indicator}'] = team_corners_matches.groupby(
        ['country', 'season', 'Team']
    )[indicator].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())

# Select only the keys and outcome columns for merging.
cols_to_merge = ['country', 'season', 'date', 'Team',
                 'SeasonPct_Over_9.5', 'Rolling5Pct_Over_9.5',
                 'SeasonPct_Over_10.5', 'Rolling5Pct_Over_10.5',
                 'SeasonPct_Over_11.5', 'Rolling5Pct_Over_11.5']

# Merge the corners outcome features into team_df.
team_df = team_df.merge(team_corners_matches[cols_to_merge],
                        on=['country', 'season', 'date', 'Team'],
                        how='left')

# =============================================================================
# 4. Process Home and Away Features for Match-Level Merging
# =============================================================================
# -- Home-Team Process --
home_subset = team_df[team_df['is_home'] == 1].copy()
home_subset.drop(columns=['Opponent'], inplace=True)
home_subset.rename(columns={'Team': 'home_team'}, inplace=True)
home_key = ['country', 'season', 'date', 'home_team', 'is_home']
# Include features starting with Overall_, Home_, SeasonPct_Over_, or Rolling5Pct_Over_
home_feats = [col for col in home_subset.columns if col not in home_key and
              (col.startswith("Overall_") or col.startswith("Home_") or
               col.startswith("SeasonPct_Over_") or col.startswith("Rolling5Pct_Over_"))]
home_features = home_subset[home_key + home_feats].copy()
def clean_home_name(col):
    return "home_" + (col[len("Home_"):] if col.startswith("Home_") else col)
home_features.rename(columns={col: clean_home_name(col) for col in home_feats}, inplace=True)

# -- Away-Team Process --
away_subset = team_df[team_df['is_home'] == 0].copy()
away_subset.drop(columns=['Opponent'], inplace=True)
away_subset.rename(columns={'Team': 'away_team'}, inplace=True)
away_key = ['country', 'season', 'date', 'away_team', 'is_home']
away_feats = [col for col in away_subset.columns if col not in away_key and
              (col.startswith("Overall_") or col.startswith("Away_") or
               col.startswith("SeasonPct_Over_") or col.startswith("Rolling5Pct_Over_"))]
away_features = away_subset[away_key + away_feats].copy()
def clean_away_name(col):
    return "away_" + (col[len("Away_"):] if col.startswith("Away_") else col)
away_features.rename(columns={col: clean_away_name(col) for col in away_feats}, inplace=True)

# =============================================================================
# 5. Merge Home and Away Features into the Match-Level DataFrame
# =============================================================================
# Start with the original match data.
match_merge_df = data.copy()
# Merge home features.
match_merge_df = match_merge_df.merge(home_features, on=['country', 'season', 'date', 'home_team'], how='left')
# Merge away features.
match_merge_df = match_merge_df.merge(away_features, on=['country', 'season', 'date', 'away_team'], how='left')

# (Optional) Display a sample.
#print(match_merge_df.head())


C:\Users\leere\AppData\Local\Temp\ipykernel_23536\209829939.py:168: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  team_df = team_df.groupby(['country', 'season', 'Team'], group_keys=False).apply(add_rolling_features_split).reset_index(drop=True)


In [10]:
match_merge_df_filter = match_merge_df[match_merge_df['home_team'] == "Chelsea"]
match_merge_df_filter

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_CornersPct_Over_5.5,away_CornersRolling5Pct_Over_5.5,away_CornersPct_Over_6.5,away_CornersRolling5Pct_Over_6.5,away_SeasonPct_Over_9.5,away_Rolling5Pct_Over_9.5,away_SeasonPct_Over_10.5,away_Rolling5Pct_Over_10.5,away_SeasonPct_Over_11.5,away_Rolling5Pct_Over_11.5
689,Eng1,25,2024-08-18,1630,1,Chelsea,Man City,0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1312,Eng1,25,2024-09-01,1330,3,Chelsea,C Palace,1,1,1,...,1.000000,1.000000,1.000000,1.000000,0.500000,0.5,0.500000,0.5,0.000000,0.0
2165,Eng1,25,2024-09-28,1500,6,Chelsea,Brighton,4,2,4,...,0.500000,0.500000,0.500000,0.500000,0.600000,0.6,0.200000,0.2,0.000000,0.0
2644,Eng1,25,2024-10-06,1400,7,Chelsea,Nottingham,1,1,0,...,0.333333,0.333333,0.333333,0.333333,0.666667,0.8,0.333333,0.4,0.166667,0.2
3349,Eng1,25,2024-10-27,1400,9,Chelsea,Newcastle,2,1,1,...,0.750000,0.750000,0.750000,0.750000,0.875000,0.8,0.750000,0.6,0.500000,0.2
4033,Eng1,25,2024-11-10,1630,11,Chelsea,Arsenal,1,1,0,...,0.400000,0.400000,0.000000,0.000000,0.800000,0.8,0.400000,0.6,0.300000,0.4
4645,Eng1,25,2024-12-01,1330,13,Chelsea,Aston Villa,3,0,2,...,0.333333,0.400000,0.333333,0.400000,0.583333,1.0,0.416667,0.8,0.166667,0.4
5284,Eng1,25,2024-12-15,1900,16,Chelsea,Brentford,2,1,1,...,0.000000,0.000000,0.000000,0.000000,0.800000,0.6,0.666667,0.4,0.533333,0.4
5584,Eng1,25,2024-12-26,1500,18,Chelsea,Fulham,1,2,1,...,0.500000,0.400000,0.375000,0.400000,0.588235,0.6,0.588235,0.6,0.352941,0.2
6041,Eng1,25,2025-01-14,1930,21,Chelsea,Bournemouth,2,2,1,...,0.600000,0.400000,0.500000,0.400000,0.650000,0.6,0.550000,0.6,0.500000,0.6


In [11]:
match_merge_df

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_CornersPct_Over_5.5,away_CornersRolling5Pct_Over_5.5,away_CornersPct_Over_6.5,away_CornersRolling5Pct_Over_6.5,away_SeasonPct_Over_9.5,away_Rolling5Pct_Over_9.5,away_SeasonPct_Over_10.5,away_Rolling5Pct_Over_10.5,away_SeasonPct_Over_11.5,away_Rolling5Pct_Over_11.5
0,Mex1,25,2024-07-05,2345,1,Puebla,Santos Laguna,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mex1,25,2024-07-06,200,1,Queretaro,Tijuana de Caliente,1,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mex1,25,2024-07-06,410,1,Juarez,Atlas,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mex1,25,2024-07-07,0,1,San Luis,Club America,2,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mex1,25,2024-07-07,200,1,G. Chivas,Toluca,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18797,Eng2,26,2025-11-26,1945,17,Blackburn,QPR,0,0,0,...,0.125000,0.0,0.125000,0.0,0.437500,0.6,0.375000,0.6,0.312500,0.6
18798,Eng2,26,2025-11-26,1945,17,Millwall,Sheffield Wed,0,0,0,...,0.142857,0.0,0.142857,0.0,0.625000,0.6,0.625000,0.6,0.437500,0.2
18799,Eng2,26,2025-11-26,1945,17,Wrexham,Bristol City,0,0,0,...,0.428571,0.6,0.428571,0.6,0.562500,0.6,0.375000,0.4,0.312500,0.2
18800,Eng4,26,2025-11-27,2000,18,Grimsby,Tranmere,0,0,0,...,0.250000,0.2,0.250000,0.2,0.470588,0.4,0.352941,0.2,0.176471,0.0


In [12]:
# ## -----------------------------
# # 1. Process Home-Team Features (with clean naming)
# # -----------------------------
# home_subset = team_df[team_df['is_home'] == 1].copy()
# home_subset = home_subset.drop(columns=['Opponent'])
# home_subset.rename(columns={'Team': 'home_team'}, inplace=True)
#
# # Key columns that remain unchanged
# home_key_cols = ['country', 'season', 'date', 'home_team', 'is_home']
#
# # Update the feature column selection to include the merged outcome columns.
# home_feature_cols = [col for col in home_subset.columns
#                      if col not in home_key_cols and
#                      (col.startswith("Overall_") or
#                       col.startswith("Home_") or
#                       col.startswith("SeasonPct_Over_") or
#                       col.startswith("Rolling5Pct_Over_"))]
#
# # Create a DataFrame with key columns and desired features
# home_features = home_subset[home_key_cols + home_feature_cols].copy()
#
# # Function to clean column names by removing any existing "Home_" prefix
# def clean_home_name(col):
#     if col.startswith("Home_"):
#         col = col[len("Home_"):]
#     return "home_" + col
#
# # Build a renaming dictionary for home features
# rename_mapping_home = {col: clean_home_name(col) for col in home_feature_cols}
# home_features.rename(columns=rename_mapping_home, inplace=True)
#
#
# # -----------------------------
# # 2. Process Away-Team Features (with clean naming)
# # -----------------------------
# away_subset = team_df[team_df['is_home'] == 0].copy()
# away_subset = away_subset.drop(columns=['Opponent'])
# away_subset.rename(columns={'Team': 'away_team'}, inplace=True)
#
# # Key columns that remain unchanged
# away_key_cols = ['country', 'season', 'date', 'away_team', 'is_home']
#
# # Update the feature column selection to include the merged outcome columns.
# away_feature_cols = [col for col in away_subset.columns
#                      if col not in away_key_cols and
#                      (col.startswith("Overall_") or
#                       col.startswith("Away_") or
#                       col.startswith("SeasonPct_Over_") or
#                       col.startswith("Rolling5Pct_Over_"))]
#
# # Create a DataFrame with key columns and desired features
# away_features = away_subset[away_key_cols + away_feature_cols].copy()
#
# # Function to clean column names by removing any existing "Away_" prefix
# def clean_away_name(col):
#     if col.startswith("Away_"):
#         col = col[len("Away_"):]
#     return "away_" + col
#
# # Build a renaming dictionary for away features
# rename_mapping_away = {col: clean_away_name(col) for col in away_feature_cols}
# away_features.rename(columns=rename_mapping_away, inplace=True)
#
#
# # -----------------------------
# # 3. Merge Processed Home- and Away-Team Features Back into the Match-Level DataFrame
# # -----------------------------
# # Start with your original match-level data
# match_df = data.copy()
#
# # Merge home features on the common keys: country, season, date, and home_team.
# match_df = match_df.merge(home_features, on=['country', 'season', 'date', 'home_team'], how='left')
#
# #Merge away features on the common keys: country, season, date, and away_team.
# match_df = match_df.merge(away_features, on=['country', 'season', 'date', 'away_team'], how='left')
#
# # # match_df now contains cleanly named columns such as "home_Rolling_GoalsScored_Mean" along with your corners outcome features.
# # print(match_df.head())


In [13]:
## -----------------------------
# 1. Process Home-Team Features (with clean naming)
# -----------------------------
home_subset = team_df[team_df['is_home'] == 1].copy()
home_subset = home_subset.drop(columns=['Opponent'])
home_subset.rename(columns={'Team': 'home_team'}, inplace=True)

# Key columns that remain unchanged
home_key_cols = ['country', 'season', 'date', 'home_team', 'is_home']

# Update the feature column selection to include the merged outcome columns.
home_feature_cols = [col for col in home_subset.columns
                     if col not in home_key_cols and
                     (col.startswith("Overall_") or
                      col.startswith("Home_") or
                      col.startswith("SeasonPct_Over_") or
                      col.startswith("Rolling5Pct_Over_"))]

# Create a DataFrame with key columns and desired features
home_features = home_subset[home_key_cols + home_feature_cols].copy()

# Function to clean column names by removing any existing "Home_" prefix
def clean_home_name(col):
    if col.startswith("Home_"):
        col = col[len("Home_"):]
    return "home_" + col

# Build a renaming dictionary for home features
rename_mapping_home = {col: clean_home_name(col) for col in home_feature_cols}
home_features.rename(columns=rename_mapping_home, inplace=True)


# -----------------------------
# 2. Process Away-Team Features (with clean naming)
# -----------------------------
away_subset = team_df[team_df['is_home'] == 0].copy()
away_subset = away_subset.drop(columns=['Opponent'])
away_subset.rename(columns={'Team': 'away_team'}, inplace=True)

# Key columns that remain unchanged
away_key_cols = ['country', 'season', 'date', 'away_team', 'is_home']

# Update the feature column selection to include the merged outcome columns.
away_feature_cols = [col for col in away_subset.columns
                     if col not in away_key_cols and
                     (col.startswith("Overall_") or
                      col.startswith("Away_") or
                      col.startswith("SeasonPct_Over_") or
                      col.startswith("Rolling5Pct_Over_"))]

# Create a DataFrame with key columns and desired features
away_features = away_subset[away_key_cols + away_feature_cols].copy()

# Function to clean column names by removing any existing "Away_" prefix
def clean_away_name(col):
    if col.startswith("Away_"):
        col = col[len("Away_"):]
    return "away_" + col

# Build a renaming dictionary for away features
rename_mapping_away = {col: clean_away_name(col) for col in away_feature_cols}
away_features.rename(columns=rename_mapping_away, inplace=True)


# -----------------------------
# 3. Merge Processed Home- and Away-Team Features Back into the Match-Level DataFrame
# -----------------------------
# Start with your original match-level data
match_df = data.copy()

# Merge home features on the common keys: country, season, date, and home_team.
match_df = match_df.merge(home_features, on=['country', 'season', 'date', 'home_team'], how='left')

#Merge away features on the common keys: country, season, date, and away_team.
match_df = match_df.merge(away_features, on=['country', 'season', 'date', 'away_team'], how='left')

# # match_df now contains cleanly named columns such as "home_Rolling_GoalsScored_Mean" along with your corners outcome features.
# print(match_df.head())


In [14]:
team_filter2 = match_df[(match_df["home_team"]=="Chelsea") | (match_df["away_team"]=="Chelsea")]
team_filter2

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_CornersPct_Over_5.5,away_CornersRolling5Pct_Over_5.5,away_CornersPct_Over_6.5,away_CornersRolling5Pct_Over_6.5,away_SeasonPct_Over_9.5,away_Rolling5Pct_Over_9.5,away_SeasonPct_Over_10.5,away_Rolling5Pct_Over_10.5,away_SeasonPct_Over_11.5,away_Rolling5Pct_Over_11.5
689,Eng1,25,2024-08-18,1630,1,Chelsea,Man City,0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
981,Eng1,25,2024-08-25,1400,2,Wolves,Chelsea,2,6,2,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1312,Eng1,25,2024-09-01,1330,3,Chelsea,C Palace,1,1,1,...,1.000000,1.000000,1.000000,1.000000,0.500000,0.500000,0.500000,0.500000,0.000000,0.000000
1517,Eng1,25,2024-09-14,2000,4,Bournemouth,Chelsea,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000
1821,Eng1,25,2024-09-21,1230,5,West Ham,Chelsea,0,3,0,...,0.000000,0.000000,0.000000,0.000000,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000
2165,Eng1,25,2024-09-28,1500,6,Chelsea,Brighton,4,2,4,...,0.500000,0.500000,0.500000,0.500000,0.600000,0.600000,0.200000,0.200000,0.000000,0.000000
2644,Eng1,25,2024-10-06,1400,7,Chelsea,Nottingham,1,1,0,...,0.333333,0.333333,0.333333,0.333333,0.666667,0.800000,0.333333,0.400000,0.166667,0.200000
2970,Eng1,25,2024-10-20,1630,8,Liverpool,Chelsea,2,1,1,...,0.000000,0.000000,0.000000,0.000000,0.571429,0.600000,0.285714,0.400000,0.142857,0.200000
3349,Eng1,25,2024-10-27,1400,9,Chelsea,Newcastle,2,1,1,...,0.750000,0.750000,0.750000,0.750000,0.875000,0.800000,0.750000,0.600000,0.500000,0.200000
3683,Eng1,25,2024-11-03,1630,10,Man Utd,Chelsea,1,1,0,...,0.000000,0.000000,0.000000,0.000000,0.555556,0.800000,0.333333,0.600000,0.111111,0.200000


In [15]:
match_df

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_CornersPct_Over_5.5,away_CornersRolling5Pct_Over_5.5,away_CornersPct_Over_6.5,away_CornersRolling5Pct_Over_6.5,away_SeasonPct_Over_9.5,away_Rolling5Pct_Over_9.5,away_SeasonPct_Over_10.5,away_Rolling5Pct_Over_10.5,away_SeasonPct_Over_11.5,away_Rolling5Pct_Over_11.5
0,Mex1,25,2024-07-05,2345,1,Puebla,Santos Laguna,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mex1,25,2024-07-06,200,1,Queretaro,Tijuana de Caliente,1,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mex1,25,2024-07-06,410,1,Juarez,Atlas,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mex1,25,2024-07-07,0,1,San Luis,Club America,2,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mex1,25,2024-07-07,200,1,G. Chivas,Toluca,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18797,Eng2,26,2025-11-26,1945,17,Blackburn,QPR,0,0,0,...,0.125000,0.0,0.125000,0.0,0.437500,0.6,0.375000,0.6,0.312500,0.6
18798,Eng2,26,2025-11-26,1945,17,Millwall,Sheffield Wed,0,0,0,...,0.142857,0.0,0.142857,0.0,0.625000,0.6,0.625000,0.6,0.437500,0.2
18799,Eng2,26,2025-11-26,1945,17,Wrexham,Bristol City,0,0,0,...,0.428571,0.6,0.428571,0.6,0.562500,0.6,0.375000,0.4,0.312500,0.2
18800,Eng4,26,2025-11-27,2000,18,Grimsby,Tranmere,0,0,0,...,0.250000,0.2,0.250000,0.2,0.470588,0.4,0.352941,0.2,0.176471,0.0


In [16]:
features = ['round', 'home_team_place_total', 'home_team_place_home', 'away_team_place_total', 'away_team_place_away', 'home_odds', 'draw_odds', 'away_odds', 'over_25_odds', 'under_25_odds', 'elo_home', 'elo_away', 'form_home', 'form_away', 'home_Overall_Rolling_GoalsScored_Mean', 'home_Overall_Rolling_GoalsScored_Std', 'home_Overall_Rolling_GoalsScored_Mean_Short', 'home_Overall_Momentum_GoalsScored', 'home_Overall_Trend_Slope_GoalsScored', 'home_Overall_Rolling_FirstHalfGoalsScored_Mean', 'home_Overall_Rolling_FirstHalfGoalsScored_Std', 'home_Overall_Rolling_FirstHalfGoalsScored_Mean_Short', 'home_Overall_Momentum_FirstHalfGoalsScored', 'home_Overall_Trend_Slope_FirstHalfGoalsScored', 'home_Overall_Rolling_Shots_Mean', 'home_Overall_Rolling_Shots_Std', 'home_Overall_Rolling_Shots_Mean_Short', 'home_Overall_Momentum_Shots', 'home_Overall_Trend_Slope_Shots', 'home_Overall_Rolling_Shots_1h_Mean', 'home_Overall_Rolling_Shots_1h_Std', 'home_Overall_Rolling_Shots_1h_Mean_Short', 'home_Overall_Momentum_Shots_1h', 'home_Overall_Trend_Slope_Shots_1h', 'home_Overall_Rolling_Corners_Mean', 'home_Overall_Rolling_Corners_Std', 'home_Overall_Rolling_Corners_Mean_Short', 'home_Overall_Momentum_Corners', 'home_Overall_Trend_Slope_Corners', 'home_Overall_Rolling_Corners_1h_Mean', 'home_Overall_Rolling_Corners_1h_Std', 'home_Overall_Rolling_Corners_1h_Mean_Short', 'home_Overall_Momentum_Corners_1h', 'home_Overall_Trend_Slope_Corners_1h', 'home_Overall_Rolling_ShotsOnTarget_Mean', 'home_Overall_Rolling_ShotsOnTarget_Std', 'home_Overall_Rolling_ShotsOnTarget_Mean_Short', 'home_Overall_Momentum_ShotsOnTarget', 'home_Overall_Trend_Slope_ShotsOnTarget', 'home_Overall_Rolling_ShotsOnTarget_1h_Mean', 'home_Overall_Rolling_ShotsOnTarget_1h_Std', 'home_Overall_Rolling_ShotsOnTarget_1h_Mean_Short', 'home_Overall_Momentum_ShotsOnTarget_1h', 'home_Overall_Trend_Slope_ShotsOnTarget_1h', 'home_Rolling_GoalsScored_Mean', 'home_Rolling_GoalsScored_Std', 'home_Rolling_GoalsScored_Mean_Short', 'home_Momentum_GoalsScored', 'home_Trend_Slope_GoalsScored', 'home_Rolling_FirstHalfGoalsScored_Mean', 'home_Rolling_FirstHalfGoalsScored_Std', 'home_Rolling_FirstHalfGoalsScored_Mean_Short', 'home_Momentum_FirstHalfGoalsScored', 'home_Trend_Slope_FirstHalfGoalsScored', 'home_Rolling_Shots_Mean', 'home_Rolling_Shots_Std', 'home_Rolling_Shots_Mean_Short', 'home_Momentum_Shots', 'home_Trend_Slope_Shots', 'home_Rolling_Shots_1h_Mean', 'home_Rolling_Shots_1h_Std', 'home_Rolling_Shots_1h_Mean_Short', 'home_Momentum_Shots_1h', 'home_Trend_Slope_Shots_1h', 'home_Rolling_Corners_Mean', 'home_Rolling_Corners_Std', 'home_Rolling_Corners_Mean_Short', 'home_Momentum_Corners', 'home_Trend_Slope_Corners', 'home_Rolling_Corners_1h_Mean', 'home_Rolling_Corners_1h_Std', 'home_Rolling_Corners_1h_Mean_Short', 'home_Momentum_Corners_1h', 'home_Trend_Slope_Corners_1h', 'home_Rolling_ShotsOnTarget_Mean', 'home_Rolling_ShotsOnTarget_Std', 'home_Rolling_ShotsOnTarget_Mean_Short', 'home_Momentum_ShotsOnTarget', 'home_Trend_Slope_ShotsOnTarget', 'home_Rolling_ShotsOnTarget_1h_Mean', 'home_Rolling_ShotsOnTarget_1h_Std', 'home_Rolling_ShotsOnTarget_1h_Mean_Short', 'home_Momentum_ShotsOnTarget_1h', 'home_Trend_Slope_ShotsOnTarget_1h', 'home_Overall_Percent_Over_1.5', 'home_Overall_Rolling5_Percent_Over_1.5', 'home_Percent_Over_1.5', 'home_Rolling5_Percent_Over_1.5', 'home_Overall_Percent_Over_2.5', 'home_Overall_Rolling5_Percent_Over_2.5', 'home_Percent_Over_2.5', 'home_Rolling5_Percent_Over_2.5', 'home_Overall_Percent_Over_3.5', 'home_Overall_Rolling5_Percent_Over_3.5', 'home_Percent_Over_3.5', 'home_Rolling5_Percent_Over_3.5', 'home_TeamPct_Over_0.5', 'home_TeamPct_Over_1.5', 'home_TeamPct_Over_2.5', 'home_TeamPct_Over_3.5', 'home_CornersPct_Over_3.5', 'home_CornersRolling5Pct_Over_3.5', 'home_CornersPct_Over_4.5', 'home_CornersRolling5Pct_Over_4.5', 'home_CornersPct_Over_5.5', 'home_CornersRolling5Pct_Over_5.5', 'home_CornersPct_Over_6.5', 'home_CornersRolling5Pct_Over_6.5', 'home_SeasonPct_Over_9.5', 'home_Rolling5Pct_Over_9.5', 'home_SeasonPct_Over_10.5', 'home_Rolling5Pct_Over_10.5', 'home_SeasonPct_Over_11.5', 'home_Rolling5Pct_Over_11.5', 'away_Overall_Rolling_GoalsScored_Mean', 'away_Overall_Rolling_GoalsScored_Std', 'away_Overall_Rolling_GoalsScored_Mean_Short', 'away_Overall_Momentum_GoalsScored', 'away_Overall_Trend_Slope_GoalsScored', 'away_Overall_Rolling_FirstHalfGoalsScored_Mean', 'away_Overall_Rolling_FirstHalfGoalsScored_Std', 'away_Overall_Rolling_FirstHalfGoalsScored_Mean_Short', 'away_Overall_Momentum_FirstHalfGoalsScored', 'away_Overall_Trend_Slope_FirstHalfGoalsScored', 'away_Overall_Rolling_Shots_Mean', 'away_Overall_Rolling_Shots_Std', 'away_Overall_Rolling_Shots_Mean_Short', 'away_Overall_Momentum_Shots', 'away_Overall_Trend_Slope_Shots', 'away_Overall_Rolling_Shots_1h_Mean', 'away_Overall_Rolling_Shots_1h_Std', 'away_Overall_Rolling_Shots_1h_Mean_Short', 'away_Overall_Momentum_Shots_1h', 'away_Overall_Trend_Slope_Shots_1h', 'away_Overall_Rolling_Corners_Mean', 'away_Overall_Rolling_Corners_Std', 'away_Overall_Rolling_Corners_Mean_Short', 'away_Overall_Momentum_Corners', 'away_Overall_Trend_Slope_Corners', 'away_Overall_Rolling_Corners_1h_Mean', 'away_Overall_Rolling_Corners_1h_Std', 'away_Overall_Rolling_Corners_1h_Mean_Short', 'away_Overall_Momentum_Corners_1h', 'away_Overall_Trend_Slope_Corners_1h', 'away_Overall_Rolling_ShotsOnTarget_Mean', 'away_Overall_Rolling_ShotsOnTarget_Std', 'away_Overall_Rolling_ShotsOnTarget_Mean_Short', 'away_Overall_Momentum_ShotsOnTarget', 'away_Overall_Trend_Slope_ShotsOnTarget', 'away_Overall_Rolling_ShotsOnTarget_1h_Mean', 'away_Overall_Rolling_ShotsOnTarget_1h_Std', 'away_Overall_Rolling_ShotsOnTarget_1h_Mean_Short', 'away_Overall_Momentum_ShotsOnTarget_1h', 'away_Overall_Trend_Slope_ShotsOnTarget_1h', 'away_Rolling_GoalsScored_Mean', 'away_Rolling_GoalsScored_Std', 'away_Rolling_GoalsScored_Mean_Short', 'away_Momentum_GoalsScored', 'away_Trend_Slope_GoalsScored', 'away_Rolling_FirstHalfGoalsScored_Mean', 'away_Rolling_FirstHalfGoalsScored_Std', 'away_Rolling_FirstHalfGoalsScored_Mean_Short', 'away_Momentum_FirstHalfGoalsScored', 'away_Trend_Slope_FirstHalfGoalsScored', 'away_Rolling_Shots_Mean', 'away_Rolling_Shots_Std', 'away_Rolling_Shots_Mean_Short', 'away_Momentum_Shots', 'away_Trend_Slope_Shots', 'away_Rolling_Shots_1h_Mean', 'away_Rolling_Shots_1h_Std', 'away_Rolling_Shots_1h_Mean_Short', 'away_Momentum_Shots_1h', 'away_Trend_Slope_Shots_1h', 'away_Rolling_Corners_Mean', 'away_Rolling_Corners_Std', 'away_Rolling_Corners_Mean_Short', 'away_Momentum_Corners', 'away_Trend_Slope_Corners', 'away_Rolling_Corners_1h_Mean', 'away_Rolling_Corners_1h_Std', 'away_Rolling_Corners_1h_Mean_Short', 'away_Momentum_Corners_1h', 'away_Trend_Slope_Corners_1h', 'away_Rolling_ShotsOnTarget_Mean', 'away_Rolling_ShotsOnTarget_Std', 'away_Rolling_ShotsOnTarget_Mean_Short', 'away_Momentum_ShotsOnTarget', 'away_Trend_Slope_ShotsOnTarget', 'away_Rolling_ShotsOnTarget_1h_Mean', 'away_Rolling_ShotsOnTarget_1h_Std', 'away_Rolling_ShotsOnTarget_1h_Mean_Short', 'away_Momentum_ShotsOnTarget_1h', 'away_Trend_Slope_ShotsOnTarget_1h', 'away_Overall_Percent_Over_1.5', 'away_Overall_Rolling5_Percent_Over_1.5', 'away_Percent_Over_1.5', 'away_Rolling5_Percent_Over_1.5', 'away_Overall_Percent_Over_2.5', 'away_Overall_Rolling5_Percent_Over_2.5', 'away_Percent_Over_2.5', 'away_Rolling5_Percent_Over_2.5', 'away_Overall_Percent_Over_3.5', 'away_Overall_Rolling5_Percent_Over_3.5', 'away_Percent_Over_3.5', 'away_Rolling5_Percent_Over_3.5', 'away_TeamPct_Over_0.5', 'away_TeamPct_Over_1.5', 'away_TeamPct_Over_2.5', 'away_TeamPct_Over_3.5', 'away_CornersPct_Over_3.5', 'away_CornersRolling5Pct_Over_3.5', 'away_CornersPct_Over_4.5', 'away_CornersRolling5Pct_Over_4.5', 'away_CornersPct_Over_5.5', 'away_CornersRolling5Pct_Over_5.5', 'away_CornersPct_Over_6.5', 'away_CornersRolling5Pct_Over_6.5', 'away_SeasonPct_Over_9.5', 'away_Rolling5Pct_Over_9.5', 'away_SeasonPct_Over_10.5', 'away_Rolling5Pct_Over_10.5', 'away_SeasonPct_Over_11.5', 'away_Rolling5Pct_Over_11.5', 'country_Arg1', 'country_Aus1', 'country_Aus2', 'country_Aut1', 'country_Bel1', 'country_Bra1', 'country_Bul1', 'country_Chi1', 'country_Chl1', 'country_Cro1', 'country_Czh1', 'country_Den1', 'country_Eng1', 'country_Eng2', 'country_Eng3', 'country_Eng4', 'country_Fra1', 'country_Fra2', 'country_Ger1', 'country_Ger2', 'country_Ger3', 'country_Gre1', 'country_Hun1', 'country_Ice1', 'country_Ire1', 'country_Isr1', 'country_Ita1', 'country_Ita2', 'country_Jap1', 'country_Jap2', 'country_Kor1', 'country_Mex1', 'country_Ned1', 'country_Ned2', 'country_Nor1', 'country_Pol1', 'country_Por1', 'country_Rom1', 'country_Sco1', 'country_Sco2', 'country_Slk1', 'country_Slo1', 'country_Spa1', 'country_Spa2', 'country_Swe1', 'country_Swe2', 'country_Swi1', 'country_Swi2', 'country_Tur1', 'country_Tur2', 'country_USA1', 'country_Arg1', 'country_Aus1', 'country_Aus2', 'country_Aut1', 'country_Bel1', 'country_Bra1', 'country_Bul1', 'country_Chi1', 'country_Chl1', 'country_Cro1', 'country_Czh1', 'country_Den1', 'country_Eng1', 'country_Eng2', 'country_Eng3', 'country_Eng4', 'country_Fra1', 'country_Fra2', 'country_Ger1', 'country_Ger2', 'country_Ger3', 'country_Gre1', 'country_Hun1', 'country_Ice1', 'country_Ire1', 'country_Isr1', 'country_Ita1', 'country_Ita2', 'country_Jap1', 'country_Jap2', 'country_Kor1', 'country_Mex1', 'country_Ned1', 'country_Ned2', 'country_Nor1', 'country_Pol1', 'country_Por1', 'country_Rom1', 'country_Sco1', 'country_Sco2', 'country_Slk1', 'country_Slo1', 'country_Spa1', 'country_Spa2', 'country_Swe1', 'country_Swe2', 'country_Swi1', 'country_Swi2', 'country_Tur1', 'country_Tur2', 'country_USA1']

In [17]:
match_df

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_CornersPct_Over_5.5,away_CornersRolling5Pct_Over_5.5,away_CornersPct_Over_6.5,away_CornersRolling5Pct_Over_6.5,away_SeasonPct_Over_9.5,away_Rolling5Pct_Over_9.5,away_SeasonPct_Over_10.5,away_Rolling5Pct_Over_10.5,away_SeasonPct_Over_11.5,away_Rolling5Pct_Over_11.5
0,Mex1,25,2024-07-05,2345,1,Puebla,Santos Laguna,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mex1,25,2024-07-06,200,1,Queretaro,Tijuana de Caliente,1,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mex1,25,2024-07-06,410,1,Juarez,Atlas,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mex1,25,2024-07-07,0,1,San Luis,Club America,2,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mex1,25,2024-07-07,200,1,G. Chivas,Toluca,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18797,Eng2,26,2025-11-26,1945,17,Blackburn,QPR,0,0,0,...,0.125000,0.0,0.125000,0.0,0.437500,0.6,0.375000,0.6,0.312500,0.6
18798,Eng2,26,2025-11-26,1945,17,Millwall,Sheffield Wed,0,0,0,...,0.142857,0.0,0.142857,0.0,0.625000,0.6,0.625000,0.6,0.437500,0.2
18799,Eng2,26,2025-11-26,1945,17,Wrexham,Bristol City,0,0,0,...,0.428571,0.6,0.428571,0.6,0.562500,0.6,0.375000,0.4,0.312500,0.2
18800,Eng4,26,2025-11-27,2000,18,Grimsby,Tranmere,0,0,0,...,0.250000,0.2,0.250000,0.2,0.470588,0.4,0.352941,0.2,0.176471,0.0


In [18]:
import function_library as fl
filtered_data = match_df[(match_df['date'].dt.date >= today) & (match_df['date'].dt.date <= end_period)].copy()
filtered_data = fl.team_name_map(filtered_data)

####
# Create a boolean mask for country == 'Chl1'
mask = (filtered_data['country'] == 'Chl1')

# Only look at those rows, and replace 'Everton' → 'Everton De Vina'
filtered_data.loc[mask, ['home_team','away_team']] = (
    filtered_data.loc[mask, ['home_team','away_team']]
    .replace('Everton', 'Everton De Vina')
)
####
filtered_data

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_CornersPct_Over_5.5,away_CornersRolling5Pct_Over_5.5,away_CornersPct_Over_6.5,away_CornersRolling5Pct_Over_6.5,away_SeasonPct_Over_9.5,away_Rolling5Pct_Over_9.5,away_SeasonPct_Over_10.5,away_Rolling5Pct_Over_10.5,away_SeasonPct_Over_11.5,away_Rolling5Pct_Over_11.5
18644,Swi1,26,2025-11-23,1300,14,Sion,FC Zurich,0,0,0,...,0.666667,0.8,0.500000,0.6,0.538462,0.6,0.384615,0.2,0.384615,0.2
18645,Rom1,26,2025-11-23,1530,17,Otelul Galati,Farul Constanta,0,0,0,...,0.875000,0.8,0.500000,0.4,0.562500,0.2,0.437500,0.0,0.312500,0.0
18646,Sco1,26,2025-11-23,1500,13,Aberdeen,Hearts,0,0,0,...,0.600000,0.6,0.600000,0.6,0.583333,0.4,0.583333,0.4,0.416667,0.4
18647,Slk1,26,2025-11-23,1700,15,Dunajska Streda,Tatran Presov,0,0,0,...,0.571429,0.6,0.428571,0.6,0.714286,0.4,0.500000,0.4,0.428571,0.2
18648,Slo1,26,2025-11-23,1630,16,NK Primorje,Mura,0,0,0,...,0.428571,0.6,0.428571,0.6,0.533333,0.6,0.466667,0.4,0.400000,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18797,Eng2,26,2025-11-26,1945,17,Blackburn,QPR,0,0,0,...,0.125000,0.0,0.125000,0.0,0.437500,0.6,0.375000,0.6,0.312500,0.6
18798,Eng2,26,2025-11-26,1945,17,Millwall,Sheff Wed,0,0,0,...,0.142857,0.0,0.142857,0.0,0.625000,0.6,0.625000,0.6,0.437500,0.2
18799,Eng2,26,2025-11-26,1945,17,Wrexham,Bristol City,0,0,0,...,0.428571,0.6,0.428571,0.6,0.562500,0.6,0.375000,0.4,0.312500,0.2
18800,Eng4,26,2025-11-27,2000,18,Grimsby,Tranmere,0,0,0,...,0.250000,0.2,0.250000,0.2,0.470588,0.4,0.352941,0.2,0.176471,0.0


In [19]:
# filtered_with_features = filtered_data[features]
# filtered_with_features

In [20]:
filtered_data[filtered_data.isna().any(axis=1)]


,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_CornersPct_Over_5.5,away_CornersRolling5Pct_Over_5.5,away_CornersPct_Over_6.5,away_CornersRolling5Pct_Over_6.5,away_SeasonPct_Over_9.5,away_Rolling5Pct_Over_9.5,away_SeasonPct_Over_10.5,away_Rolling5Pct_Over_10.5,away_SeasonPct_Over_11.5,away_Rolling5Pct_Over_11.5
18644,Swi1,26,2025-11-23,1300,14,Sion,FC Zurich,0,0,0,...,0.666667,0.8,0.500000,0.6,0.538462,0.6,0.384615,0.2,0.384615,0.2
18645,Rom1,26,2025-11-23,1530,17,Otelul Galati,Farul Constanta,0,0,0,...,0.875000,0.8,0.500000,0.4,0.562500,0.2,0.437500,0.0,0.312500,0.0
18646,Sco1,26,2025-11-23,1500,13,Aberdeen,Hearts,0,0,0,...,0.600000,0.6,0.600000,0.6,0.583333,0.4,0.583333,0.4,0.416667,0.4
18647,Slk1,26,2025-11-23,1700,15,Dunajska Streda,Tatran Presov,0,0,0,...,0.571429,0.6,0.428571,0.6,0.714286,0.4,0.500000,0.4,0.428571,0.2
18648,Slo1,26,2025-11-23,1630,16,NK Primorje,Mura,0,0,0,...,0.428571,0.6,0.428571,0.6,0.533333,0.6,0.466667,0.4,0.400000,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18797,Eng2,26,2025-11-26,1945,17,Blackburn,QPR,0,0,0,...,0.125000,0.0,0.125000,0.0,0.437500,0.6,0.375000,0.6,0.312500,0.6
18798,Eng2,26,2025-11-26,1945,17,Millwall,Sheff Wed,0,0,0,...,0.142857,0.0,0.142857,0.0,0.625000,0.6,0.625000,0.6,0.437500,0.2
18799,Eng2,26,2025-11-26,1945,17,Wrexham,Bristol City,0,0,0,...,0.428571,0.6,0.428571,0.6,0.562500,0.6,0.375000,0.4,0.312500,0.2
18800,Eng4,26,2025-11-27,2000,18,Grimsby,Tranmere,0,0,0,...,0.250000,0.2,0.250000,0.2,0.470588,0.4,0.352941,0.2,0.176471,0.0


In [21]:
# filtered_data.dropna(inplace=True)
# filtered_data

In [22]:
import os
import glob
import pandas as pd
from joblib import load

# ── CONFIG ────────────────────────────────────────────────────────────────
NEW_MODEL_DIR = r"C:\Users\leere\PycharmProjects\Football_ML3\Goals\Under_2_5\model_file"
fixtures_df   = filtered_data.copy()   # your fresh fixtures DataFrame
IMPORT_DIR    = r"C:\Users\leere\OneDrive\Desktop\IMPORTS"
OUT_PATH      = os.path.join(IMPORT_DIR, "u25_predictions.csv")

os.makedirs(IMPORT_DIR, exist_ok=True)

# ── HELPERS ───────────────────────────────────────────────────────────────
def newest_pkl(model_dir: str) -> str:
    """Pick the most recently modified best_model_*.pkl."""
    pkls = glob.glob(os.path.join(model_dir, "best_model_*_calibrated_*.pkl"))
    if not pkls:
        raise RuntimeError(f"No PKLs found in {model_dir}")
    return max(pkls, key=os.path.getmtime)

def align_features(df_in: pd.DataFrame, feature_contract: list[str]) -> pd.DataFrame:
    """
    Align df to the saved training feature list:
      - add missing columns as 0
      - drop extras
      - preserve column order
    """
    X = df_in.reindex(columns=feature_contract, fill_value=0)
    # ensure numeric dtypes where possible
    for c in X.columns:
        if pd.api.types.is_object_dtype(X[c]):
            X[c] = pd.to_numeric(X[c], errors="ignore")
    return X

# ── LOAD MODEL ────────────────────────────────────────────────────────────
pkl_path = newest_pkl(NEW_MODEL_DIR)
md = load(pkl_path)
model     = md['model']          # calibrated estimator
threshold = float(md.get('threshold', 0.5))
feat_list = list(md['features']) # feature contract used in training

# Optional: if training used one-hot on 'country', recreate those columns
if 'country' in fixtures_df.columns and any(f.startswith('country_') for f in feat_list):
    fixtures_df = pd.get_dummies(fixtures_df, columns=['country'], prefix='country')

# ── PREPARE FEATURES ─────────────────────────────────────────────────────
X = align_features(fixtures_df, feat_list)

# Drop rows with missing feature values; keep row alignment
X = X.dropna()
fixtures_df = fixtures_df.loc[X.index].copy()

# ── PREDICT ───────────────────────────────────────────────────────────────
if hasattr(model, "predict_proba"):
    proba = model.predict_proba(X)
    proba = proba[:, 1] if proba.ndim == 2 else proba
    mask  = proba >= threshold
else:
    # Fallback (unlikely): use predict -> bool
    mask  = model.predict(X).astype(bool)
    proba = pd.Series(mask, index=X.index, dtype=float)

positives = fixtures_df.loc[mask].copy()
positives["pred_proba_u25"] = proba[mask]
positives = positives.sort_values("pred_proba_u25", ascending=False)  # optional, nice to have

# ── WRITE IMPORT FILE ─────────────────────────────────────────────────────
if positives.empty:
    print(f"No selections (≥ {threshold:.2f}); nothing to write.")
else:
    # Build your import structure (adjust columns if your fixtures use different names)
    if not {"home_team", "away_team"}.issubset(positives.columns):
        raise KeyError("fixtures_df must contain 'home_team' and 'away_team' columns.")

    out_df = pd.DataFrame({
        'EventName':     positives['home_team'] + ' v ' + positives['away_team'],
        'Provider':      'under_2_5_goals',
        'MarketName':    'Over/Under 2.5 Goals',
        'SelectionName': 'Under 2.5 Goals',
        #'PredProb':      positives['pred_proba_u25'].round(4),
        #'Threshold':     threshold,
    })

    out_df.to_csv(OUT_PATH, index=False)
    print(f"✓ Wrote {len(out_df)} U2.5 selections to:\n   {OUT_PATH}\n   (model: {pkl_path})")


✓ Wrote 6 U2.5 selections to:
   C:\Users\leere\OneDrive\Desktop\IMPORTS\u25_predictions.csv
   (model: C:\Users\leere\PycharmProjects\Football_ML3\Goals\Under_2_5\model_file\best_model_xgb_calibrated_20250818_175932.pkl)


In [23]:
# See ALL fixtures with probabilities + bet flag (and also save positives)
import os
import glob
import pandas as pd
from joblib import load

# ── CONFIG ────────────────────────────────────────────────────────────────
NEW_MODEL_DIR = r"C:\Users\leere\PycharmProjects\Football_ML3\Goals\Under_2_5\model_file"
fixtures_df   = filtered_data.copy()   # your fresh fixtures DataFrame
IMPORT_DIR    = r"C:\Users\leere\OneDrive\Desktop\IMPORTS"
OUT_ALL_PATH  = os.path.join(IMPORT_DIR, "u25_fixtures_with_probs.csv")
OUT_POS_PATH  = os.path.join(IMPORT_DIR, "u25_predictions.csv")

os.makedirs(IMPORT_DIR, exist_ok=True)

# ── HELPERS ───────────────────────────────────────────────────────────────
def newest_pkl(model_dir: str) -> str:
    """Pick the most recently modified best_model_*.pkl."""
    pkls = glob.glob(os.path.join(model_dir, "best_model_*_calibrated_*.pkl"))
    if not pkls:
        raise RuntimeError(f"No PKLs found in {model_dir}")
    return max(pkls, key=os.path.getmtime)

def align_features(df_in: pd.DataFrame, feature_contract: list[str]) -> pd.DataFrame:
    """
    Align df to the saved training feature list:
      - add missing columns as 0
      - drop extras
      - preserve column order
      - coerce numerics when possible
    """
    X = df_in.reindex(columns=feature_contract, fill_value=0)
    for c in X.columns:
        if pd.api.types.is_object_dtype(X[c]):
            X[c] = pd.to_numeric(X[c], errors="ignore")
    return X

# ── LOAD MODEL ────────────────────────────────────────────────────────────
pkl_path = newest_pkl(NEW_MODEL_DIR)
md        = load(pkl_path)
model     = md["model"]                 # calibrated estimator
threshold = float(md.get("threshold", 0.5))
feat_list = list(md["features"])        # training feature contract

# If training used one-hot on 'country', recreate those dummies
if "country" in fixtures_df.columns and any(f.startswith("country_") for f in feat_list):
    fixtures_df = pd.get_dummies(fixtures_df, columns=["country"], prefix="country")

# ── SCORE ALL FIXTURES ────────────────────────────────────────────────────
X = align_features(fixtures_df, feat_list)
X = X.dropna()
scored = fixtures_df.loc[X.index].copy()

if hasattr(model, "predict_proba"):
    proba = model.predict_proba(X)
    proba = proba[:, 1] if proba.ndim == 2 else proba
else:
    # Rare fallback
    proba = model.predict(X).astype(float)

scored["pred_proba_u25"] = proba
scored["threshold"] = threshold
scored["bet"] = scored["pred_proba_u25"] >= threshold

# Optional convenience columns
if {"home_team", "away_team"}.issubset(scored.columns):
    scored["EventName"] = scored["home_team"] + " v " + scored["away_team"]

scored["Provider"]      = "under_2_5_goals"
scored["MarketName"]    = "Over/Under 2.5 Goals"
scored["SelectionName"] = "Under 2.5 Goals"

# Sort by probability (nice to read)
scored = scored.sort_values("pred_proba_u25", ascending=False).reset_index(drop=True)

# Put key columns first if they exist
preferred_first = [
    "EventName", "date", "league", "country", "home_team", "away_team",
    "Provider", "MarketName", "SelectionName", "pred_proba_o25", "threshold", "bet"
]
cols = [c for c in preferred_first if c in scored.columns] + \
       [c for c in scored.columns if c not in preferred_first]

# ── WRITE CSVs ────────────────────────────────────────────────────────────
# scored[cols].to_csv(OUT_ALL_PATH, index=False)
# print(f"✓ Wrote ALL fixtures with probabilities to:\n   {OUT_ALL_PATH}")
#
# positives = scored[scored["bet"]].copy()
# positives[cols].to_csv(OUT_POS_PATH, index=False)
# print(f"✓ Wrote positive O2.5 selections to:\n   {OUT_POS_PATH}")
#
# # In Jupyter, preview the top rows:
# try:
#     from IPython.display import display
#     display(scored[cols].head(20))
# except Exception:
#     print(scored[cols].head(20))


In [25]:
scored

,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,away_goals_ht,...,country_Swi1,country_Tur1,country_Tur2,pred_proba_u25,threshold,bet,EventName,Provider,MarketName,SelectionName
0,25,2025-11-27,2330,36,Fluminense,Sao Paulo,0,0,0,0,...,False,False,False,0.901460,0.63,True,Fluminense v Sao Paulo,under_2_5_goals,Over/Under 2.5 Goals,Under 2.5 Goals
1,26,2025-11-27,2000,18,Grimsby,Tranmere,0,0,0,0,...,False,False,False,0.901460,0.63,True,Grimsby v Tranmere,under_2_5_goals,Over/Under 2.5 Goals,Under 2.5 Goals
2,26,2025-11-24,1945,15,Reims,Montpellier,0,0,0,0,...,False,False,False,0.745354,0.63,True,Reims v Montpellier,under_2_5_goals,Over/Under 2.5 Goals,Under 2.5 Goals
3,26,2025-11-23,1515,15,Huesca,Sporting Gijon,0,0,0,0,...,False,False,False,0.717175,0.63,True,Huesca v Sporting Gijon,under_2_5_goals,Over/Under 2.5 Goals,Under 2.5 Goals
4,26,2025-11-25,1900,11,Bastia,Laval,0,0,0,0,...,False,False,False,0.643915,0.63,True,Bastia v Laval,under_2_5_goals,Over/Under 2.5 Goals,Under 2.5 Goals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,26,2025-11-23,1500,16,FC Nordsjaelland,Fredericia,0,0,0,0,...,False,False,False,0.279720,0.63,False,FC Nordsjaelland v Fredericia,under_2_5_goals,Over/Under 2.5 Goals,Under 2.5 Goals
152,26,2025-11-23,1430,11,RB Leipzig,Werder Bremen,0,0,0,0,...,False,False,False,0.279720,0.63,False,RB Leipzig v Werder Bremen,under_2_5_goals,Over/Under 2.5 Goals,Under 2.5 Goals
153,26,2025-11-25,1900,17,RKC Waalwijk,MVV Maastricht,0,0,0,0,...,False,False,False,0.279720,0.63,False,RKC Waalwijk v MVV Maastricht,under_2_5_goals,Over/Under 2.5 Goals,Under 2.5 Goals
154,26,2025-11-25,1900,17,ADO Den Haag,De Graafschap,0,0,0,0,...,False,False,False,0.279720,0.63,False,ADO Den Haag v De Graafschap,under_2_5_goals,Over/Under 2.5 Goals,Under 2.5 Goals
